In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 30
generations = int(3000/necoc)
generations

100

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
7


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:726
number of class2:874

 ****** Running generation 0 ****** 

Population's average fitness: 0.50599 stdev: 0.05325
Best fitness: 0.61000 - size: (2, 3) - species 7 - id 175
Average adjusted fitness: 0.122
Mean genetic distance 2.593, standard deviation 0.457
Population of 202 members in 12 species
Total extinctions: 0
Generation time: 3.263 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51644 stdev: 0.05248
Best fitness: 0.61000 - size: (2, 3) - species 3 - id 175
Average adjusted fitness: 0.120
Mean genetic distance 2.760, standard deviation 0.545
Population of 201 members in 14 species
Total extinctions: 0
Generation time: 3.661 sec (3.462 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.52634 stdev: 0.05110
Best fitness: 0.61687 - size: (2, 3) - species 6 - id 477
Average adjusted fitness: 0.112
Mean genetic distance 2.828, standard deviation 0.583
Population of 201 members in 20 species
Total

Population's average fitness: 0.54310 stdev: 0.05864
Best fitness: 0.63438 - size: (7, 13) - species 22 - id 1977
Average adjusted fitness: 0.153
Mean genetic distance 3.192, standard deviation 0.583
Population of 205 members in 23 species
Total extinctions: 0
Generation time: 5.330 sec (5.092 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.54048 stdev: 0.05833
Best fitness: 0.63438 - size: (7, 13) - species 22 - id 1977
Average adjusted fitness: 0.147
Mean genetic distance 3.195, standard deviation 0.574
Population of 199 members in 23 species
Total extinctions: 0
Generation time: 5.853 sec (5.201 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.53975 stdev: 0.06008
Best fitness: 0.63438 - size: (7, 13) - species 22 - id 1977
Average adjusted fitness: 0.129
Mean genetic distance 3.177, standard deviation 0.566
Population of 194 members in 23 species
Total extinctions: 0
Generation time: 5.644 sec (5.278 average)

 ******

Population's average fitness: 0.54947 stdev: 0.06067
Best fitness: 0.64062 - size: (8, 15) - species 5 - id 4418
Average adjusted fitness: 0.151
Mean genetic distance 3.143, standard deviation 0.570
Population of 196 members in 18 species
Total extinctions: 0
Generation time: 5.829 sec (5.854 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.54804 stdev: 0.05920
Best fitness: 0.64062 - size: (8, 15) - species 5 - id 4418
Average adjusted fitness: 0.136
Mean genetic distance 3.095, standard deviation 0.572
Population of 198 members in 17 species
Total extinctions: 0
Generation time: 6.002 sec (5.892 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.54048 stdev: 0.06062
Best fitness: 0.64062 - size: (8, 15) - species 5 - id 4418
Average adjusted fitness: 0.132
Mean genetic distance 3.155, standard deviation 0.580
Population of 204 members in 17 species
Total extinctions: 0
Generation time: 6.212 sec (5.929 average)

 ****** Ru

Population's average fitness: 0.54682 stdev: 0.05797
Best fitness: 0.64250 - size: (7, 17) - species 55 - id 10741
Average adjusted fitness: 0.154
Mean genetic distance 3.325, standard deviation 0.571
Population of 202 members in 26 species
Total extinctions: 0
Generation time: 6.572 sec (6.400 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.54987 stdev: 0.05648
Best fitness: 0.64938 - size: (8, 19) - species 52 - id 11323
Average adjusted fitness: 0.119
Mean genetic distance 3.309, standard deviation 0.575
Population of 204 members in 26 species
Total extinctions: 0
Generation time: 6.780 sec (6.364 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.54671 stdev: 0.05847
Best fitness: 0.64938 - size: (8, 19) - species 52 - id 11323
Average adjusted fitness: 0.134
Mean genetic distance 3.292, standard deviation 0.571
Population of 193 members in 24 species
Total extinctions: 0
Generation time: 6.877 sec (6.412 average)

 ***

Mean genetic distance 3.241, standard deviation 0.598
Population of 197 members in 14 species
Total extinctions: 0
Generation time: 6.967 sec (7.294 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.55264 stdev: 0.05980
Best fitness: 0.65875 - size: (16, 34) - species 70 - id 15149
Average adjusted fitness: 0.159
Mean genetic distance 3.252, standard deviation 0.580
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 6.990 sec (7.260 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.54310 stdev: 0.06251
Best fitness: 0.65875 - size: (16, 34) - species 70 - id 15149
Average adjusted fitness: 0.153
Mean genetic distance 3.239, standard deviation 0.579
Population of 197 members in 15 species
Total extinctions: 0
Generation time: 7.137 sec (7.200 average)

 ****** Running generation 99 ****** 

Population's average fitness: 0.54564 stdev: 0.06174
Best fitness: 0.65875 - size: (16, 34) - species 70 - id 

...

Population's average fitness: 0.53909 stdev: 0.08397
Best fitness: 0.62375 - size: (3, 5) - species 1 - id 252
Average adjusted fitness: 0.152
Mean genetic distance 2.795, standard deviation 0.567
Population of 200 members in 17 species
Total extinctions: 0
Generation time: 3.141 sec (3.010 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.54729 stdev: 0.08203
Best fitness: 0.62375 - size: (3, 5) - species 7 - id 252
Average adjusted fitness: 0.137
Mean genetic distance 2.793, standard deviation 0.609
Population of 199 members in 17 species
Total extinctions: 0
Generation time: 3.313 sec (3.111 average)

 ****** Running generation 3 ****** 

Population's average fitness: 0.55162 stdev: 0.07996
Best fitness: 0.62375 - size: (3, 5) - species 11 - id 252
Average adjusted fitness: 0.142
Mean genetic distance 2.790, standard deviation 0.593
Population of 201 members in 17 species
Total extinctions: 0
Generation time: 3.474 sec (3.202 average)

 ****** Running g

Population's average fitness: 0.56198 stdev: 0.07563
Best fitness: 0.63562 - size: (3, 6) - species 24 - id 1681
Average adjusted fitness: 0.175
Mean genetic distance 3.167, standard deviation 0.603
Population of 201 members in 20 species
Total extinctions: 0
Generation time: 4.943 sec (5.009 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.55924 stdev: 0.07719
Best fitness: 0.63562 - size: (3, 6) - species 24 - id 1681
Average adjusted fitness: 0.177
Mean genetic distance 3.151, standard deviation 0.592
Population of 202 members in 23 species
Total extinctions: 0
Generation time: 4.969 sec (5.026 average)

 ****** Running generation 27 ****** 

Population's average fitness: 0.56692 stdev: 0.07083
Best fitness: 0.63562 - size: (3, 6) - species 24 - id 1681
Average adjusted fitness: 0.173
Mean genetic distance 3.133, standard deviation 0.602
Population of 192 members in 22 species
Total extinctions: 0
Generation time: 5.099 sec (4.916 average)

 ****** Ru

Generation time: 6.054 sec (5.739 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.54509 stdev: 0.08632
Best fitness: 0.64875 - size: (3, 7) - species 11 - id 4981
Average adjusted fitness: 0.145
Mean genetic distance 3.118, standard deviation 0.547
Population of 202 members in 21 species
Total extinctions: 0
Generation time: 6.151 sec (5.833 average)

 ****** Running generation 51 ****** 

Population's average fitness: 0.56485 stdev: 0.07544
Best fitness: 0.64875 - size: (3, 7) - species 11 - id 4981
Average adjusted fitness: 0.180
Mean genetic distance 3.144, standard deviation 0.545
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 6.139 sec (5.913 average)

 ****** Running generation 52 ****** 

Population's average fitness: 0.55547 stdev: 0.07973
Best fitness: 0.64875 - size: (3, 7) - species 11 - id 4981
Average adjusted fitness: 0.154
Mean genetic distance 3.165, standard deviation 0.563
Population of 203 members in 21 

Population's average fitness: 0.55095 stdev: 0.08610
Best fitness: 0.65000 - size: (9, 18) - species 11 - id 11658
Average adjusted fitness: 0.155
Mean genetic distance 3.052, standard deviation 0.567
Population of 203 members in 17 species
Total extinctions: 0
Generation time: 7.041 sec (6.935 average)

 ****** Running generation 75 ****** 

Population's average fitness: 0.55326 stdev: 0.08463
Best fitness: 0.65000 - size: (9, 18) - species 11 - id 11658
Average adjusted fitness: 0.174
Mean genetic distance 3.032, standard deviation 0.590
Population of 198 members in 16 species
Total extinctions: 0
Generation time: 7.344 sec (7.001 average)

 ****** Running generation 76 ****** 

Population's average fitness: 0.55327 stdev: 0.08444
Best fitness: 0.65000 - size: (9, 18) - species 11 - id 11658
Average adjusted fitness: 0.157
Mean genetic distance 3.064, standard deviation 0.588
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 6.816 sec (6.980 average)

 ***

Population's average fitness: 0.56513 stdev: 0.08087
Best fitness: 0.65625 - size: (12, 23) - species 59 - id 14105
Average adjusted fitness: 0.187
Mean genetic distance 3.042, standard deviation 0.546
Population of 195 members in 16 species
Total extinctions: 0
Generation time: 8.061 sec (7.711 average)

 ****** Running generation 99 ****** 

Population's average fitness: 0.56232 stdev: 0.08323
Best fitness: 0.65625 - size: (12, 23) - species 59 - id 14105
Average adjusted fitness: 0.170
Mean genetic distance 3.053, standard deviation 0.543
Population of 196 members in 16 species
Total extinctions: 0
Generation time: 7.913 sec (7.781 average)


In [9]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.32

In [11]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.380625
Avg Base Learner:0.7065208333333332


In [12]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:486 Total connections::1868
